На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [239]:
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

import plotly.express as px

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

from google_drive_downloader import GoogleDriveDownloader as gdd

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [240]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [241]:
gdd.download_file_from_google_drive(file_id='1gZ5kV2TdLn1-HppknO20i4RkwJzhM2eE',
                                    dest_path='./data/ABTest.xlsx',
                                    unzip=False)
df = pd.read_excel('./data/ABTest.xlsx')

In [242]:
df.head(10)

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
5,2380,variant,0.0
6,2849,control,0.0
7,9168,control,0.0
8,6205,variant,0.0
9,7548,control,0.0


In [243]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [244]:
df.shape

(10000, 3)

In [245]:
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [246]:
df.isna().sum()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

In [247]:
df.groupby('VARIANT_NAME')['USER_ID'].count().reset_index()

,VARIANT_NAME,USER_ID
0,control,4984
1,variant,5016


In [248]:
#подсчет кол-во вхождений пользователей в тестовые группуы
variants = df.groupby('USER_ID', as_index = False).agg({'VARIANT_NAME': pd.Series.nunique})

In [249]:
#очистка групп от дубликатов
df = df[~df.USER_ID.isin(variants.query('VARIANT_NAME > 1').USER_ID)].copy(deep = True)

In [250]:
df.groupby('VARIANT_NAME')['REVENUE'].describe().reset_index()

,VARIANT_NAME,count,mean,std,min,25%,50%,75%,max
0,control,3026.0,0.155506,3.708620,0.0,0.0,0.0,0.0,196.01
1,variant,3044.0,0.058909,0.760344,0.0,0.0,0.0,0.0,23.04


In [251]:
fig = px.histogram(df,
                   x='REVENUE',
                   color = 'VARIANT_NAME',
                   title='averge revenue',
                   marginal = 'box',
                   nbins = 50,
                   barmode='overlay')

fig.show()

In [256]:
control = df[df['VARIANT_NAME'] == 'control'].copy(deep = True)
variant = df[df['VARIANT_NAME'] == 'variant'].copy(deep = True)
continious_result(control, variant, column = 'REVENUE')

100%|██████████| 10000/10000 [00:05<00:00, 1773.42it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.03608,0.159864,0.710771,"[-0.001, 0.26]",0.097432
